In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
#data=pd.read_csv('language translation/fra.txt',sep='/n',encoding='utf-8')

C:\Users\ajay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
#data.iloc[0]

Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)    Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: ...
Name: 0, dtype: object

In [33]:
file=open('language translation/fra.txt','r')
lines=file.read().split('\n')

In [5]:
lines[100000].split('\t')

['She takes care of my children.',
 "Elle s'occupe de mes enfants.",
 'CC-BY 2.0 (France) Attribution: tatoeba.org #309970 (CK) & #181079 (Romira)']

In [6]:
num_samples=20000

In [17]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
for line in lines[:num_samples]:
    input_text,target_text,_=line.split('\t')
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            

In [18]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

In [19]:
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

In [20]:
num_encoder_tokens

76

In [21]:
max_encoder_seq_length=max([len(i) for i in input_texts])
max_decoder_seq_length=max([len(i) for i in target_texts])

In [22]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 76
Number of unique output tokens: 99
Max sequence length for inputs: 18
Max sequence length for outputs: 63


In [23]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [24]:
## ONE HOT REPRESENTATION
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(target_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(target_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')


In [25]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [26]:
from tensorflow.keras.layers import Dense,LSTM,Input
from tensorflow.keras.models import Model,Sequential

In [27]:
latent_dim=256

In [45]:
## model for train time



encoded_inputs=Input(shape=(None,num_encoder_tokens))
encoder_lstm=LSTM(latent_dim, return_state=True)
_,state_h,state_c=encoder_lstm(encoded_inputs)
encoded_states=[state_h,state_c]

decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoded_states)

decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)


model=Model(inputs=[encoded_inputs,decoder_inputs],outputs=decoder_outputs)

In [46]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, None, 76)]   0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, None, 99)]   0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 256), (None, 340992      input_17[0][0]                   
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, None, 256),  364544      input_18[0][0]                   
                                                                 lstm_8[0][1]               

In [76]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=20,
          validation_split=0.3)

Train on 14000 samples, validate on 6000 samples
Epoch 1/20
14000/14000 [==============================] - 207s 15ms/sample - loss: 0.2351 - accuracy: 0.9277 - val_loss: 0.4414 - val_accuracy: 0.8718
Epoch 2/20
14000/14000 [==============================] - 137s 10ms/sample - loss: 0.2263 - accuracy: 0.9306 - val_loss: 0.4386 - val_accuracy: 0.8730
Epoch 3/20
14000/14000 [==============================] - 137s 10ms/sample - loss: 0.2191 - accuracy: 0.9326 - val_loss: 0.4414 - val_accuracy: 0.8733
Epoch 4/20
14000/14000 [==============================] - 136s 10ms/sample - loss: 0.2126 - accuracy: 0.9347 - val_loss: 0.4436 - val_accuracy: 0.8734
Epoch 5/20
14000/14000 [==============================] - 136s 10ms/sample - loss: 0.2063 - accuracy: 0.9365 - val_loss: 0.4478 - val_accuracy: 0.8734
Epoch 6/20
14000/14000 [==============================] - 141s 10ms/sample - loss: 0.2002 - accuracy: 0.9384 - val_loss: 0.4467 - val_accuracy: 0.8742
Epoch 7/20
14000/14000 [=====================

In [78]:
## model during test time


encoder_model=Model(encoded_inputs,encoded_states)

decoder_state_h=Input(shape=(latent_dim,))
decoder_state_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_h,decoder_state_c]

decoder_output,decoder_states_h,decoder_states_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoded_states_outputs=[decoder_states_h,decoder_states_c]

decoder_outputs=decoder_dense(decoder_output)

decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoded_states_outputs)












In [79]:



reverse_input_tokens=dict([(i,j) for i,j in enumerate(input_characters)])
reverse_output_tokens=dict([(i,j) for i,j in enumerate(target_characters)])


def decode_sequence(input_sequence):
    state_value=encoder_model.predict(input_sequence)
    
    target_seq=np.zeros([1,1,num_decoder_tokens])
    
    target_seq[0,0,target_token_index['\t']]==1
    
    
    flag=0
    decode_seq=''
    
    while flag==0:
        output_tokens,h,c=decoder_model.predict([target_seq]+state_value)
        sample_token_index=np.argmax(output_tokens[0,-1,:])
        sample_char=reverse_output_tokens[sample_token_index]
        decode_seq+=sample_char
        
        
        if sample_char=='\n' or len(decode_seq)>=max_decoder_seq_length:
            flag=1
            
        target_seq=np.zeros([1,1,num_decoder_tokens])
        
        target_seq[0,0,target_token_index[sample_char]]=1
        
        state_value=[h,c]
        
    return decode_seq






for seq_index in range(5000,5005):

    # Take one sequence (part of the training set)

    # for trying out decoding.

    input_seq = encoder_input_data[seq_index: seq_index + 1]

    decoded_sentence = decode_sequence(input_seq)
    
    print('-')

    print('Input sentence:', input_texts[seq_index])
    print('target setence:',target_texts[seq_index])

    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Can you pitch?
target setence: 	Sais-tu donner le ton ?

Decoded sentence: Savez-vous faire du ski ?

-
Input sentence: Can you pitch?
target setence: 	Savez-vous lancer ?

Decoded sentence: Savez-vous faire du ski ?

-
Input sentence: Can you pitch?
target setence: 	Savez-vous donner le ton ?

Decoded sentence: Savez-vous faire du ski ?

-
Input sentence: Can you skate?
target setence: 	Sais-tu patinerâ€¯?

Decoded sentence: Savez-vous faire du ski ?

-
Input sentence: Cats are cute.
target setence: 	Les chats sont mignons.

Decoded sentence: Les chats sont mignons.

